In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

## Fibonacci numbers

Defined iteratively as sum of two previous numbers in sequence:

$$x_{n+1} = x_n + x_{n-1} $$

Giving sequence:

1,1,2,3,5,8,13,...

Following is some interesting ideas for speeding up their computation by considering this sequence in terms of linear algebra:
https://www.youtube.com/watch?v=5WMiEYIeF8s&list=PLdkTDauaUnQpzuOCZyUUZc0lxf4-PXNR5&index=6

In [3]:
# simple way to compute using recusion
# but having to recompute REALLY slows us down!
def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

tic = time.time()
fib_n = fib(35)
toc = time.time()

dt = toc-tic
print(f"value:{fib_n}, time: {dt:.3g} [seconds]")

value:14930352, time: 3.03 [seconds]


In [16]:
# speed up by caching previously used values
# can use array of values
# or implicitly using pythons functools.lru_cache annotations

from functools import lru_cache

@lru_cache
def fib_cached(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib_cached(n-1) + fib_cached(n-2)
    
tic = time.time()
fib_n = fib_cached(500)
toc = time.time()

dt = toc-tic
print(f"value:{fib_n:3g}, time: {dt:.3g} [seconds]")

value:2.25592e+104, time: 0.000621 [seconds]


Instead of caching can instead consider the sequence elements as vectors:

$$\begin{pmatrix}x_{n+1} \\x_{n} \\\end{pmatrix} =\begin{pmatrix}1 & 1 \\1 & 0 \\\end{pmatrix}\begin{pmatrix}x_{n} \\x_{n-1} \\\end{pmatrix} $$

Thus starting at the first two elements, any element $x_{n+1}$ can be computed by repeated application of the matrix:

$$\begin{pmatrix}x_{n+1} \\x_{n} \\\end{pmatrix} =\begin{pmatrix}1 & 1 \\1 & 0 \\\end{pmatrix}^n\begin{pmatrix}1 \\1 \\\end{pmatrix} $$

This matrix power can be computed faster by continually squaring matrices to get to the suitable power, making the runtime logarithmic

In [35]:
def fib_matrix(n):
    
    A = np.array([[1,1],[1,0]])
    A_n = np.linalg.matrix_power(A,n)
    x = A_n @ np.array([1,1])
    
    return x[1]

tic = time.time()
fib_n = fib_matrix(1000)
toc = time.time()

dt = toc-tic
print(f"value:{fib_n:3g}, time: {dt:.3g} [seconds]")

value:9.07957e+18, time: 0.000203 [seconds]


Finally - near constant runtime can be obtained by using the eigen decomposition, which allows the matrix 

$$A = \begin{pmatrix} 1 & 1 \\1 & 0 \\\end{pmatrix}$$

To be raised to the power, $A^n$ by using:

$$A = VDV^{-1}$$

with 

$$A^n = (VDV^{-1})^n = VD^nV{-1}$$

Thus allowing a near constant time computation of Fibonacci numbers!

In [29]:
A = np.array([[1,1],[1,0]])
# eigen decomposition:
D,V = np.linalg.eig(A)
invV = np.linalg.inv(V)
# check correctness of decomposition
Ap = V @ (np.diag(D) @ invV)
print(Ap)

[[ 1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00 -1.11022302e-16]]


In [34]:
def fib_eig(n):
    Dn = np.power(D,n)
    An = V@(np.diag(Dn)@invV)
    x = An @ np.array([1,1])
    return x[1]

tic = time.time()
fib_n = fib_eig(1000)
toc = time.time()

dt = toc-tic
print(f"value:{fib_n:3g}, time: {dt:.3g} [seconds]")

value:7.03304e+208, time: 0.000338 [seconds]
